In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../../src')
from ecomplexity import ecomplexity

# 小数点以下 桁数 6
pd.options.display.float_format = '{:.3f}'.format


In [2]:
import initial_condition
from process import weight


In [3]:
global data_dir, filter_dir, output_dir
data_dir = '../../data/interim/internal/filtered_before_agg/'
filter_dir = '../../data/interim/internal/filter_after_agg/'
output_dir = '../../data/interim/internal/filtered_after_agg/'


In [4]:
# 初期条件
ar = initial_condition.AR
year_style = initial_condition.YEAR_STYLE

year_start = initial_condition.YEAR_START
year_end = initial_condition.YEAR_END
year_range = initial_condition.YEAR_RANGE

extract_population = initial_condition.EXTRACT_POPULATION
top_p_or_num = initial_condition.TOP_P_OR_NUM
region_corporation = initial_condition.REGION_CORPORATION
applicant_weight = initial_condition.APPLICANT_WEIGHT

classification = initial_condition.CLASSIFICATION
classification = 'ipc3'
class_weight = initial_condition.CLASS_WEIGHT

filter_condition = f'{ar}_{year_style}_{extract_population}_reg_num_top_{top_p_or_num[0]}_{top_p_or_num[1]}_{region_corporation}'
output_condition = f'{ar}_{year_style}_{extract_population}_{top_p_or_num[0]}_{top_p_or_num[1]}_{region_corporation}_{applicant_weight}_{classification}_{class_weight}'



In [5]:
# 全体
all_df = pd.read_csv(f'{data_dir}japan.csv', 
                     encoding='utf-8', 
                     sep=',', 
                     usecols=['reg_num', 
                              region_corporation, 
                              f'{ar}_{year_style}', 
                              f'{classification}'], 
                     dtype={'reg_num': str, 
                            region_corporation: str, 
                            f'{ar}_{year_style}': np.int64, 
                            f'{classification}': str})

all_df = all_df[all_df[f'{ar}_{year_style}'].isin(range(year_start, year_end+1))]\
               .drop_duplicates()\
# display(all_df.head())


# 各期間
sep_year_df_dict = {}

for year in range(year_start, year_end+1, year_range):
    sep_year_df_dict[f'{year}-{year+year_range-1}'] = all_df[all_df[f'{ar}_{year_style}'].isin(range(year, year+year_range))]


ValueError: Usecols do not match columns, columns expected but not found: ['ipc3']

In [6]:
# 特許分類による重みづけ
# 全体
if class_weight == 'fraction':
    all_df = weight.by_classification(all_df, region_corporation, classification)
elif class_weight == 'duplication':
    all_df['class_weight'] = 1
all_df[f'{ar}_{year_style}_period'] = f'{year_start}-{year_end}'


# 期間ごと
# sep_year_df_dict = {}
# sep_year_reg_num_top_df_dict = {}
for period, sep_year_df in sep_year_df_dict.items():
    if class_weight == 'fraction':
        sep_year_df_dict[period] = weight.by_classification(sep_year_df, region_corporation, classification)
    elif class_weight == 'duplication':
        sep_year_df_dict[period] = sep_year_df.groupby([region_corporation, classification])[['reg_num']].nunique().reset_index(drop=False)
    sep_year_df_dict[period][f'{ar}_{year_style}_period'] = period

# 共同出願の重みづけ
# 全体
if applicant_weight == 'fraction':
    all_df = weight.by_applicant(all_df, region_corporation)
elif applicant_weight == 'duplication':
    all_df['applicant_weight'] = 1
all_df[f'{ar}_{year_style}_period'] = f'{year_start}-{year_end}'


# 期間ごと
# sep_year_df_dict = {}
# sep_year_reg_num_top_df_dict = {}
for period, sep_year_df in sep_year_df_dict.items():
    if applicant_weight == 'fraction':
        sep_year_df_dict[period] = weight.by_applicant(sep_year_df, region_corporation)
    elif applicant_weight == 'duplication':
        sep_year_df_dict[period]['applicant_weight'] = 1
    sep_year_df_dict[period][f'{ar}_{year_style}_period'] = period



In [7]:
all_reg_num_df = all_df.copy()
all_reg_num_df['reg_num'] = 1 / all_reg_num_df['class_weight'] / all_reg_num_df['applicant_weight']
all_reg_num_df = all_reg_num_df.groupby([f'{ar}_{year_style}_period', region_corporation, classification])[['reg_num']]\
                               .sum().reset_index(drop=False)\
                               .sort_values(['reg_num'], ascending=[False])
sep_year_reg_num_df_dict = sep_year_df_dict.copy()
for period, sep_year_reg_num_df in sep_year_reg_num_df_dict.items():
    sep_year_reg_num_df['reg_num'] = 1 / sep_year_reg_num_df['class_weight'] / sep_year_reg_num_df['applicant_weight']
    sep_year_reg_num_df = sep_year_reg_num_df.groupby([f'{ar}_{year_style}_period', region_corporation, classification])[['reg_num']]\
                                             .sum().reset_index(drop=False)\
                                             .sort_values(['reg_num'], ascending=[False])
    sep_year_reg_num_df_dict[period] = sep_year_reg_num_df
sep_year_reg_num_df = pd.concat([sep_year_reg_num_df for sep_year_reg_num_df in sep_year_reg_num_df_dict.values()], axis='index', ignore_index=True)
sep_year_reg_num_df


,app_nendo_period,right_person_name,schmoch35,reg_num
0,1981-1990,パナソニツクホールデイングス株式会社,1,9345.286
1,1981-1990,パナソニツクホールデイングス株式会社,2,8878.486
2,1981-1990,キヤノン株式会社,9,7256.333
3,1981-1990,富士通株式会社,6,6360.683
4,1981-1990,コニカミノルタ株式会社,9,5765.250
...,...,...,...,...
231802,2001-2010,大旺新洋株式会社,31,0.071
231803,2001-2010,九州電力株式会社,31,0.071
231804,2001-2010,東北電力株式会社,9,0.071
231805,2001-2010,日本電計株式会社,31,0.071


In [9]:
# フィルタリング
reg_num_filter_df = pd.read_csv(f'{filter_dir}{filter_condition}.csv',
                                encoding='utf-8',
                                sep=',', 
                                usecols=[f'{ar}_{year_style}_period', region_corporation],
                                dtype=str)
reg_num_filter_df

,app_nendo_period,right_person_name
0,1981-2010,パナソニツクホールデイングス株式会社
1,1981-2010,キヤノン株式会社
2,1981-2010,株式会社東芝
3,1981-2010,日本電気株式会社
4,1981-2010,三菱電機株式会社
...,...,...
1933,1981-2010,モリタ宮田工業株式会社
1934,1981-2010,スピードフアム株式会社
1935,1981-2010,株式会社ＩＨＩ検査計測
1936,1981-2010,株式会社ＪＣＵ


In [10]:

if extract_population == 'all':
    all_reg_num_top_df = pd.merge(
        all_reg_num_df,
        reg_num_filter_df,
        on=[f'{ar}_{year_style}_period', region_corporation],
        how='inner',
    )
    # sep_year_reg_num_top_df = pd.merge(
    #     sep_year_reg_num_df,
    #     reg_num_filter_df[[region_corporation]],
    #     on=[region_corporation], 
    #     how='inner'
    # )
    sep_year_reg_num_top_df = sep_year_reg_num_df[sep_year_reg_num_df[region_corporation].isin(reg_num_filter_df[region_corporation])]
sep_year_reg_num_top_df

reg_num_top_df = pd.concat([all_reg_num_top_df, sep_year_reg_num_top_df], 
                           axis='index', ignore_index=True)
reg_num_top_df


,app_nendo_period,right_person_name,schmoch35,reg_num
0,1981-2010,キヤノン株式会社,9,23723.000
1,1981-2010,キヤノン株式会社,28,9996.333
2,1981-2010,キヤノン株式会社,2,9525.167
3,1981-2010,キヤノン株式会社,6,7450.333
4,1981-2010,キヤノン株式会社,3,6352.500
...,...,...,...,...
105713,2001-2010,五洋建設株式会社,31,0.071
105714,2001-2010,四国電力株式会社,9,0.071
105715,2001-2010,北陸電力株式会社,9,0.071
105716,2001-2010,九州電力株式会社,31,0.071


In [11]:
reg_num_top_df.to_csv(f'{output_dir}{output_condition}.csv', 
                      encoding='utf-8', 
                      sep=',', 
                      index=False)


In [12]:
output_condition

'app_nendo_all_p_3_right_person_name_fraction_schmoch35_fraction'